In [1]:
import urllib.request
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from ipywidgets import widgets, interactive

In [2]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

## Legendary Hero Analysis - Are they actually "Legendary"?

One new feature that was introduced into the game in the last year was the addition of "Legendary Heroes". These heroes, aside from a few originally from FE Heroes, are renditions of some of the most popular/powerful characters in the franchise with some changes compared to the original character. Some units change movement type, weapon type, or even both, while also gaining unique skills and personalized weapons. From the title of being a "Legendary Hero", one would think that these characters are the best of the best. However, is this the case based solely on their stats? 

First, we'll start off by looking at the list of legendary characters:

In [3]:
heroes = pd.read_csv("hero_data.csv")

In [4]:
heroes.head()

,Name,Origin,Rarities,Story/TT/GHB?,Release Date,Legendary,Color,Weapon Type,Movement Type,HP,Attack,Speed,Defense,Resistance,Total
0,Abel: The Panther,Fire Emblem: Mystery of the Emblem,"[3, 4]",None,2017-02-02,False,Blue,Lance,Cavalry,39,33,32,25,25,154
1,Alfonse: Prince of Askr,Fire Emblem Heroes,[4],Story,2017-02-02,False,Red,Sword,Infantry,43,35,25,32,22,157
2,Alfonse: Spring Prince,Fire Emblem Heroes,[5],None,2018-03-22,False,Green,Axe,Cavalry,41,35,33,30,18,157
3,Alm: Hero of Prophecy,Fire Emblem Echoes: Shadows of Valentia,[5],None,2017-04-14,False,Red,Sword,Infantry,45,33,30,28,22,158
4,Alm: Imperial Ascent,Fire Emblem Echoes: Shadows of Valentia,[5],None,2019-08-16,False,Red,Sword,Infantry,42,38,35,37,20,172


In [5]:
legend = heroes[heroes["Legendary"] == True]
legend

,Name,Origin,Rarities,Story/TT/GHB?,Release Date,Legendary,Color,Weapon Type,Movement Type,HP,Attack,Speed,Defense,Resistance,Total
5,Alm: Saint-King,Fire Emblem Echoes: Shadows of Valentia,[5],None,2019-04-27,True,Colorless,Bow,Infantry,40,37,37,26,22,162
20,Azura: Vallite Songstress,Fire Emblem Fates,[5],None,2018-12-28,True,Blue,Tome,Flying,33,30,35,17,24,139
95,Eirika: Graceful Resolve,Fire Emblem: The Sacred Stones,[5],None,2018-10-31,True,Red,Sword,Cavalry,38,31,39,27,21,156
103,Eliwood: Blazing Knight,Fire Emblem: The Blazing Blade,[5],None,2019-06-28,True,Red,Sword,Cavalry,37,36,34,33,16,156
107,Ephraim: Legendary Lord,Fire Emblem: The Sacred Stones,[5],None,2018-02-28,True,Blue,Lance,Cavalry,43,36,27,33,18,157
123,Fjorm: Princess of Ice,Fire Emblem Heroes,[5],None,2017-11-28,True,Blue,Lance,Infantry,39,30,31,30,34,164
141,Gunnthrá: Voice of Dreams,Fire Emblem Heroes,[5],None,2017-12-28,True,Green,Tome,Cavalry,37,31,33,19,25,145
153,Hector: Marquess of Ostia,Fire Emblem: The Blazing Blade,[5],None,2018-06-28,True,Green,Axe,Armored,47,40,23,38,26,174
163,Hríd: Icy Blade,Fire Emblem Heroes,[5],None,2018-11-28,True,Red,Sword,Cavalry,41,37,17,36,26,157
169,Ike: Vanguard Legend,Fire Emblem: Radiant Dawn,[5],None,2018-01-31,True,Red,Sword,Infantry,41,36,30,35,21,163


### Compared to Other Versions of the Same Character:

When looking at some of the legendary heroes compared to alternate versions of themselves, some seem to have significantly higher stats:

In [6]:
heroes[heroes['Name'].str.contains("Tiki")][["Name", "Release Date", "Legendary", "Color", "Weapon Type", 
                   "Movement Type", "HP", "Attack", "Speed", "Defense", "Resistance", "Total"]]

,Name,Release Date,Legendary,Color,Weapon Type,Movement Type,HP,Attack,Speed,Defense,Resistance,Total
22,Bantu: Tiki's Guardian,2019-09-19,False,Red,Breath,Infantry,45,34,28,35,26,168
410,Tiki: Beachside Scion,2018-07-10,False,Red,Breath,Flying,40,33,37,32,26,168
411,Tiki: Dragon Scion,2017-02-02,False,Red,Breath,Infantry,41,31,30,32,29,163
412,Tiki: Legendary Dragon,2018-09-28,True,Blue,Breath,Armored,45,34,35,37,29,180
413,Tiki: Naga's Voice,2017-02-02,False,Red,Breath,Infantry,40,35,23,35,24,157
414,Tiki: Summering Scion,2017-06-30,False,Green,Axe,Infantry,35,36,30,32,24,157
415,Tiki: Torpid Dragon,2019-05-10,False,Colorless,Breath,Armored,45,36,34,37,28,180


Some seem to have noticeably lower stats:

In [7]:
heroes[heroes['Name'].str.contains("Azura")][["Name", "Release Date", "Legendary", "Color", "Weapon Type", 
                   "Movement Type", "HP", "Attack", "Speed", "Defense", "Resistance", "Total"]]

,Name,Release Date,Legendary,Color,Weapon Type,Movement Type,HP,Attack,Speed,Defense,Resistance,Total
17,Azura: Celebratory Spirit,2017-12-31,False,Green,Axe,Flying,37,30,35,22,26,150
18,Azura: Lady of Ballads,2017-09-29,False,Green,Axe,Infantry,35,32,34,20,28,149
19,Azura: Lady of the Lake,2017-02-02,False,Blue,Lance,Infantry,36,31,33,21,28,149
20,Azura: Vallite Songstress,2018-12-28,True,Blue,Tome,Flying,33,30,35,17,24,139
21,Azura: Young Songstress,2018-11-12,False,Green,Tome,Infantry,36,33,29,17,25,140


And some of them seem to have near identical statlines as other versions:

In [9]:
heroes[heroes['Name'].str.contains("Hector")][["Name", "Release Date", "Legendary", "Color", "Weapon Type", 
                   "Movement Type", "HP", "Attack", "Speed", "Defense", "Resistance", "Total"]]

,Name,Release Date,Legendary,Color,Weapon Type,Movement Type,HP,Attack,Speed,Defense,Resistance,Total
149,Hector: Brave Warrior,2018-08-21,False,Blue,Lance,Armored,48,37,22,39,28,174
150,Hector: Dressed-Up Duo,2019-10-09,False,Red,Sword,Armored,47,41,26,39,27,180
151,Hector: General of Ostia,2017-02-02,False,Green,Axe,Armored,52,36,24,37,19,168
152,Hector: Just Here to Fight,2018-02-09,False,Green,Axe,Armored,50,38,21,40,24,173
153,Hector: Marquess of Ostia,2018-06-28,True,Green,Axe,Armored,47,40,23,38,26,174


Thus, it is not very useful to compare legendary units with other versions of said character. One thing to note is that legendary heroes often showcase a customized design and build compared to their original counterparts. For a number of the legendary heroes, they use a different weapon type and may not even be of the same movement type. Hence, may be more useful to compare them with other units of like weapon and like movement type.

We will look to compare the stats of these characters with those of similar weapon and move type to see where they fall in the distribution of characters.

In [10]:
# Make drop down for comparison of unit composition

legend_comp = widgets.Dropdown(
    options = legend["Name"].unique(),
    value = legend["Name"].unique()[0],
    description = "Name:",
)

stat_comp = widgets.Dropdown(
    options=["HP", "Attack", "Speed", "Defense", "Resistance"],
    value='HP',
    description='Stat:',
)

def plotsit(leg, stat):
    plt.figure(figsize=(8, 5))
    leghero = legend[legend["Name"] == leg]
    display(leghero[["Name", "Origin", "Release Date", "Color", "Weapon Type", "Movement Type",
                   "HP", "Attack", "Speed", "Defense", "Resistance", "Total"]])
    tab = heroes[heroes["Weapon Type"] == leghero["Weapon Type"].iloc[0]]
    tab2 = tab[tab["Movement Type"] == leghero["Movement Type"].iloc[0]]
    plt.axvline(leghero[stat].item(), color='r', linestyle='dashed', linewidth=1)
    #plt.hist(tab2[stat], bins = max(tab2[stat]) - min(tab2[stat]) + 1)
    #plt.xticks(np.arange(min(tab2[stat]), max(tab2[stat]) + 1))
    tab2[stat].hist(bins = np.arange(min(tab2[stat]), max(tab2[stat]) + 1, 1), 
                    range = np.arange(min(tab2[stat]), max(tab2[stat]) + 1, 1), align = "left")
    mi = min(tab2[stat])
    ma = max(tab2[stat])
    #plt.xticks(np.arange(mi, ma + 1, 1))
    plt.show()

interactive(plotsit, leg = legend_comp, stat = stat_comp)

interactive(children=(Dropdown(description='Name:', options=('Alm: Saint-King', 'Azura: Vallite Songstress', '…

We will also create a dataframe to detail the percentile of each hero's stats in their respective distribution.

In [11]:
legend_perc = pd.DataFrame(columns = ["Name", "HP", "Attack", "Speed", "Defense", "Resistance", "Total"])

for char in legend["Name"].unique():
    legend_perc = legend_perc.append({'Name': char, 'HP': 0, 'Attack': 0, 'Speed': 0, 'Defense': 0, 'Resistance': 0,
                                     'Total': 0}, ignore_index=True)
    for stat in ["HP", "Attack", "Speed", "Defense", "Resistance", "Total"]:
        perc = stats.percentileofscore(heroes[stat], legend[legend["Name"].str.contains(char)][stat].item(), 
                                       kind = "weak")
        legend_perc.loc[legend_perc["Name"] == char, stat] = perc

legend_perc

,Name,HP,Attack,Speed,Defense,Resistance,Total
0,Alm: Saint-King,59.0909,92.2727,92.7273,47.5,34.0909,77.2727
1,Azura: Vallite Songstress,3.63636,20.4545,75.6818,12.9545,45.4545,0.909091
2,Eirika: Graceful Resolve,38.8636,29.0909,99.0909,52.2727,27.5,49.7727
3,Eliwood: Blazing Knight,32.0455,87.0455,65.9091,80,3.63636,49.7727
4,Ephraim: Legendary Lord,82.0455,87.0455,25.6818,80,11.8182,63.8636
5,Fjorm: Princess of Ice,46.3636,20.4545,40,66.1364,94.7727,83.1818
6,Gunnthrá: Voice of Dreams,32.0455,29.0909,52.9545,19.0909,52.5,8.86364
7,Hector: Marquess of Ostia,96.1364,98.8636,12.5,97.5,59.3182,97.0455
8,Hríd: Icy Blade,68.4091,92.2727,1.59091,92.5,59.3182,63.8636
9,Ike: Vanguard Legend,68.4091,87.0455,36.3636,89.0909,27.5,82.0455


If you look carefully at the above dataframe, you can see that there is an immenese range in the statistical prowess for legendary characters. One of the characters is tied for the highest stat total (Tiki), meanwhile another is barely above the 1st percentile (Azura). However, many would agree that Azura is one of the more useful characters available, despite her low stat total (the GamePress Offensive Tier List has her at the highest tier).

However, only four of the sixteen currently-available legendary heroes are below average in terms of total stats, so there appears to be some correlation.

One other interesting note is that only six of the 16 are below the 78th percentile for speed, possibly an indicator of the importance of that particular in the game (which, in my experience, is the case). At least some of those six have unique skills that account for their lack of speed.

From what we see above, what makes a legendary hero "legendary" cannot be seen merely by their stat distribution. Personalized weapons, skills tied to particular legendary heroes, and in-game roles are just some of the factors that are likely to help distinguish these heroes from the rest of the field (aside from grandiose designs). More analysis will be made into that another time.